In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import csv,os,re
import sklearn
import numpy as np
import pickle
import random
import gensim

C:\Users\Mudit\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [4]:
#Build word embeddings model from cluster-wise corpus. 
#Get vector representation for the n-nearest neighbouring points in the same cluster as the target point and use cosine similarity

In [5]:
#Loading the artist-level k-means clustering model:

In [45]:
artist_model = pickle.load(open('artist-level-clustering-model','rb'))

In [46]:
df = pd.read_csv(r'Data/lyrics_normalized.csv')

In [47]:
song_ind = random.randint(0,len(df))

In [48]:
song = df.iloc[song_ind]

In [49]:
#Order of features to take: polarity,ttr,unique_word_count,word density
vals = np.array(list([song['Sentiment Polarity'],song['Type-Token Ratio'],song['Unique_Word_Count'],song['Word Density']]))

In [50]:
song['Unique_Word_Count']

0.6507936507936508

In [51]:
vals

array([-0.05232975,  0.54028436,  0.65079365,  0.83059468])

In [73]:
prediction = artist_model.predict(vals.reshape(1,4))[0]

In [62]:
hasattr(artist_model.labels_,'next')

False

In [67]:
from gensim.models import Word2Vec

In [77]:
#Generate four corpuses - one for each cluster
corpus_one = ''
corpus_two = ''
corpus_three = ''
corpus_four = ''


In [85]:
for ind,cluster in enumerate(artist_model.labels_):
    if cluster == 0:
        corpus_one += df.iloc[ind]['lyrics']
    elif cluster == 1:
        corpus_two += df.iloc[ind]['lyrics']
    elif cluster == 2:
        corpus_three += df.iloc[ind]['lyrics']
    else:
        corpus_four += df.iloc[ind]['lyrics']

In [90]:
#Build word embeddings for each cluster:


In [91]:
embedding_one = Word2Vec(corpus_one,size = 100,min_count = 5)

C:\Users\Mudit\Anaconda3\envs\tensorflow_gpuenv\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [97]:
pickle.dump(embedding_one,open('Pickled models/artist_embedding_one','wb'))

In [92]:
embedding_two = Word2Vec(corpus_two,size = 100,min_count = 5)

In [98]:
pickle.dump(embedding_two,open('Pickled models/artist_embedding_two','wb'))

In [93]:
embedding_three = Word2Vec(corpus_three,size = 100,min_count = 5)

In [99]:
pickle.dump(embedding_three,open('Pickled models/artist_embedding_three','wb'))

In [95]:
embedding_four = Word2Vec(corpus_four,size = 100,min_count = 5)

In [100]:
pickle.dump(embedding_four,open('Pickled models/artist_embedding_four','wb'))